# Exploratory Data Analysis -- Summarizing the Data's Main Characteristics with Visualizations  
### *HMDA Data for 2015 -- Simple Random Sample of 25,000 Tuples*  
--  
This notebook explores the Home Mortgage Disclosure Act (HMDA) data for one year -- 2015.

---  


In [2]:
# Importing Libraries.

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math

import os
import psycopg2
import pandas.io.sql as psql
import sqlalchemy
from sqlalchemy import create_engine

from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
from scipy import stats
from pylab import*
from matplotlib.ticker import LogLocator

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [5]:
# Connecting to AWS PostgreSQL RDS.

engine_hmda_2017 = psycopg2.connect(
    database="paddle_loan_canoe",
    user="reporting_user",
    password="team_loan_canoe2019",
    host="aws-pgsql-loan-canoe.cr3nrpkvgwaj.us-east-2.rds.amazonaws.com",
    port='5432'
)

In [ ]:
#  Reading the HMDA 2017 dataset; join population and education datasets appropriately for 2017 
#  for the first 50,000 rows -- as a dataframe using pandas: df.

df = pd.read_sql_query ('''SELECT * From ''', cnx)

# Using pandas to view the first 5 rows (NB: why does it start at 0?).
df.head(5)